In [ ]:
# Install dependencies
!pip install -q diffusers transformers accelerate gradio safetensors xformers

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
import gradio as gr

In [ ]:
# Load the model (Dreamlike Photoreal 2.0)
model_id = "dreamlike-art/dreamlike-photoreal-2.0"  # You can replace with a sharper model

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    use_auth_token=True,  # Replace with True if using private models
    safety_checker=None
)
pipe.to("cuda")
pipe.enable_xformers_memory_efficient_attention()

In [ ]:
def generate(prompt, guidance=7.5, steps=30, width=512, height=512):
    image = pipe(prompt, guidance_scale=guidance, num_inference_steps=steps, height=height, width=width).images[0]
    return image

In [ ]:
# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("# 🎨 Dreamlike Stable Diffusion Generator")
    with gr.Row():
        prompt = gr.Textbox(label="Prompt", placeholder="Enter your dream...")
        image_output = gr.Image(label="Generated Image")
    with gr.Row():
        guidance = gr.Slider(1, 20, value=7.5, label="Guidance Scale")
        steps = gr.Slider(10, 50, value=30, label="Inference Steps")
    with gr.Row():
        width = gr.Slider(256, 768, step=64, value=512, label="Width")
        height = gr.Slider(256, 768, step=64, value=512, label="Height")
    generate_btn = gr.Button("Generate")
    
    generate_btn.click(fn=generate, inputs=[prompt, guidance, steps, width, height], outputs=image_output)

demo.launch(debug=True)